In [ ]:
#=====================================================
#Complete municipal-level pipeline (poverty + housing + R²)
#Fecha: 2026.01.07
#=====================================================

In [1]:
#!/usr/bin/env python
# coding: utf-8

import h5py
import numpy as np
import pandas as pd
from scipy.sparse import coo_matrix
from sklearn.metrics import r2_score

In [ ]:
# =====================================================
# 0. DATE
# =====================================================
DATE_TAG = "2026.01.07"

In [ ]:
# =====================================================
# 1. INPUT PATHS
# =====================================================
H5_FILE   = "data/full_cell_occupancy.h5"
CSV_PRED  = "data/predicciones_ensemble_600k_12K_GOOD.csv"
CSV_POP   = "data/INEGI_CPV2020_n9_.csv"
CSV_CONEVAL = "data/coneval_indicadores_2010_2015_2020b.csv"

In [ ]:
# =====================================================
# 2. HDF5 INSPECTION
# =====================================================
print("\n🔎 INSPECCIÓN DEL HDF5\n")
with h5py.File(H5_FILE, "r") as f:
    for k in f.keys():
        d = f[k]
        print(f"{k:15s} | shape={d.shape} | ndim={d.ndim} | dtype={d.dtype}")


🔎 INSPECCIÓN DEL HDF5

cell_codigo     | shape=(695754, 1) | ndim=2 | dtype=object
col             | shape=(744123, 1) | ndim=2 | dtype=uint32
mun_code        | shape=(2457, 1) | ndim=2 | dtype=object
row             | shape=(744123, 1) | ndim=2 | dtype=uint32
shape           | shape=(2, 1) | ndim=2 | dtype=uint32
val             | shape=(744123, 1) | ndim=2 | dtype=float32


In [ ]:
# =====================================================
# 3. HDF5 BASE READING
# =====================================================
with h5py.File(H5_FILE, "r") as f:
    cell_codigo = f["cell_codigo"][:].ravel()
    row = f["row"][:].ravel().astype(np.int64) - 1
    col = f["col"][:].ravel().astype(np.int64) - 1
    val = f["val"][:].ravel()
    mun_code = f["mun_code"][:].ravel()
    shape = f["shape"][:].ravel()

cell_codigo = np.array([x.decode("utf-8") for x in cell_codigo])
mun_code    = np.array([x.decode("utf-8") for x in mun_code])

n_cells, n_mun = int(shape[0]), int(shape[1])
n_nnz = len(row)

print("\n DIMENSIONS")
print("Cells       :", n_cells)
print("Municipalities:", n_mun)
print("NNZ         :", n_nnz)


📐 DIMENSIONES
Celdas     : 695754
Municipios : 2457
NNZ        : 744123


In [ ]:
# =====================================================
# 4. BASE CELL DATAFRAME
# =====================================================
df_cells = pd.DataFrame({
    "cell_idx": np.arange(n_cells),
    "cell_codigo": cell_codigo
})


In [ ]:
# =====================================================
# 5. ADD PREDICTIONS (PER NNZ CELL)
# =====================================================
df_pred = pd.read_csv(CSV_PRED)
df_pred["codigo"] = df_pred["codigo"].astype(str)

df_cells = df_cells.merge(
    df_pred[["codigo", "pred_ensemble"]],
    left_on="cell_codigo",
    right_on="codigo",
    how="left"
).drop(columns="codigo")

pred_cell = df_cells["pred_ensemble"].to_numpy()
pred_nnz = pred_cell[row]

In [ ]:
# =====================================================
# 6. ADD INEGI POPULATION (PER NNZ CELL)
# =====================================================
df_pop = pd.read_csv(CSV_POP)
df_pop["CODIGO"] = df_pop["CODIGO"].astype(str)
df_pop["POBTOT"] = pd.to_numeric(df_pop["POBTOT"], errors="coerce").fillna(0)

df_cells = df_cells.merge(
    df_pop[["CODIGO", "POBTOT"]],
    left_on="cell_codigo",
    right_on="CODIGO",
    how="left"
).drop(columns="CODIGO")

df_cells["POBTOT"] = df_cells["POBTOT"].fillna(0).astype(np.float32)
pop_cell = df_cells["POBTOT"].to_numpy()
pop_nnz = pop_cell[row]


In [ ]:
# =====================================================
# 7. WRITE pred_ensemble AND POBTOT TO HDF5
# =====================================================
with h5py.File(H5_FILE, "a") as f:
    for key in ["pred_ensemble", "POBTOT"]:
        if key in f:
            del f[key]

    f.create_dataset("pred_ensemble", data=pred_nnz.astype(np.float32), compression="gzip")
    f.create_dataset("POBTOT", data=pop_nnz.astype(np.float32), compression="gzip")

print("\n pred_ensemble and POBTOT added to the HDF5")


✅ pred_ensemble y POBTOT agregados al HDF5


In [ ]:
# =====================================================
# 8. MUNICIPAL AGGREGATION OF POVERTY (MODEL)
# =====================================================
num_mat = coo_matrix(
    (val * pred_nnz * pop_nnz, (row, col)),
    shape=(n_cells, n_mun)
)
den_mat = coo_matrix(
    (val * pop_nnz, (row, col)),
    shape=(n_cells, n_mun)
)

num = np.asarray(num_mat.sum(axis=0)).ravel()
p_total_model = np.asarray(den_mat.sum(axis=0)).ravel()

pred_municipal = np.zeros(n_mun, dtype=np.float32)
mask = p_total_model > 0
pred_municipal[mask] = num[mask] / p_total_model[mask]

df_poverty = pd.DataFrame({
    "mun_code": mun_code,
    "pred_municipal": pred_municipal,
    "p_total": p_total_model
})

df_poverty = df_poverty[df_poverty["p_total"] > 0].reset_index(drop=True)

OUT_POVERTY = f"poverty_municipal_aggregated_{DATE_TAG}.csv"
df_poverty.to_csv(OUT_POVERTY, index=False)
print(f"{OUT_POVERTY} saved")

📄 poverty_municipal_aggregated_2026.01.07.csv guardado


In [ ]:
# =====================================================
# 9. MUNICIPAL HOUSING DEPRIVATION (REAL, 0–1)
# =====================================================
df_coneval = pd.read_csv(CSV_CONEVAL)

df_coneval["cve_mun"] = df_coneval["cve_mun"].astype(str).str.zfill(5)
df_coneval["poblacion_2020"] = pd.to_numeric(
    df_coneval["poblacion_2020"], errors="coerce"
).fillna(0)
df_coneval["calidad_vivienda_2020"] = pd.to_numeric(
    df_coneval["calidad_vivienda_2020"], errors="coerce"
).fillna(0)

grouped = df_coneval.groupby("cve_mun")

personas_carencia = grouped["calidad_vivienda_2020"].sum()
p_total_real = grouped["poblacion_2020"].sum()

carencia_norm = personas_carencia / p_total_real

df_vivienda = pd.DataFrame({
    "cve_mun": carencia_norm.index,
    "carencia_vivienda_2020": carencia_norm.values,
    "p_total": p_total_real.values
})

df_vivienda = df_vivienda[df_vivienda["p_total"] > 0].reset_index(drop=True)

OUT_VIVIENDA = f"municipal_housing_deprivation_2020_{DATE_TAG}.csv"
df_vivienda.to_csv(OUT_VIVIENDA, index=False)
print(f"{OUT_VIVIENDA} saved")

📄 carencia_vivienda_municipal_2020_2026.01.07.csv guardado


In [ ]:
# =====================================================
# 10. R² CALCULATION (REAL vs PRED)
# =====================================================
df_poverty["cve_mun"] = (
    df_poverty["mun_code"].astype(str).str.replace("MX", "", regex=False).str.zfill(5)
)

df_compare = df_vivienda.merge(
    df_poverty[["cve_mun", "pred_municipal"]],
    on="cve_mun",
    how="inner"
)

y_true = df_compare["carencia_vivienda_2020"].values
y_pred = df_compare["pred_municipal"].values

r2 = r2_score(y_true, y_pred)

print("\n FINAL RESULT")
print("Comparable municipalities:", len(df_compare))
print("R² =", r2)

OUT_COMPARE = f"comparison_real_vs_pred_municipal_{DATE_TAG}.csv"
df_compare.to_csv(OUT_COMPARE, index=False)
print(f" {OUT_COMPARE} saved")

print("\n COMPLETE")


📊 RESULTADO FINAL
Municipios comparables: 2456
R² = 0.5028472412726145
📄 comparison_real_vs_pred_municipal_2026.01.07.csv guardado

🎉 PIPELINE COMPLETO FINALIZADO
